In [1]:
from pulp import *
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger, lpSum, value, LpBinary,LpStatusOptimal
import pulp
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", message="Overwriting previously set objective.")
import utility
import docplex.mp.model
import docplex
import docplex_explainer
import mymetrics
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import joblib
import dill
from alibi.explainers import AnchorTabular


In [2]:
def load_data(dataset_name):
    if dataset_name == 'Iris':
        # Load Dataset
        dataset = datasets.load_iris()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        # Scale
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        # Check if binary targets
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Banknote':
        # Load Dataset
        df = pd.read_csv('./datasets/banknote_authentication.csv')
        # Scale
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Blood_Transfusion':
        df = pd.read_csv('./datasets/blood_transfusion.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Breast_Cancer':
        dataset = datasets.load_breast_cancer()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Climate':
        df = pd.read_csv('./datasets/climate_model_simulation_crashes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Glass':
        df = pd.read_csv('./datasets/glass.csv')
        df['target'] = df['target'].apply(lambda x: 1 if x in [1, 2, 3] else 0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Ionosphere':
        df = pd.read_csv('./datasets/ionosphere.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Modeling':
        df = pd.read_csv('./datasets/User_Knowledge_Modeling.csv')
        df['target'] = df['target'].apply(lambda x: 1 if x == 'Low' else 0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Parkinsons':
        df = pd.read_csv('./datasets/parkinsons.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Pima':
        df = pd.read_csv('./datasets/diabetes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Sonar':
        df = pd.read_csv('./datasets/sonar.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Wine':
        dataset = datasets.load_wine()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Vertebral-Column':
        dataset_name = 'Vertebral-Column'
        df = pd.read_csv('./datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])
        df['target']=np.where(df['target']=='AB',1,0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
        
    else:
        print("Incorrect dataset name")

def parse_explanation(explanation, feature_names, epsilon=1e-6):
    bounds = [[0, 1] for _ in range(len(feature_names))]
    conditions = explanation

    for condition in conditions:
        condition_no_space = condition.replace(' ', '')  # for regex matching
        # Check for double inequality
        match = re.match(r'(\d+\.?\d*)\s*(<|<=)\s*([^\s<>]+)\s*(<|<=)\s*(\d+\.?\d*)', condition_no_space)
        
        if match:
            value_1, op1, feature_token, op2, value_2 = match.groups()
            value_1 = float(value_1)
            value_2 = float(value_2)
            lower_bound = value_1 if op1 == '<=' else value_1 + epsilon
            upper_bound = value_2 if op2 == '<=' else value_2
            upper_bound = upper_bound if op2 == '<=' else upper_bound - epsilon

            for idx, feature in enumerate(feature_names):
                if feature.replace(" ", "") in feature_token:
                    bounds[idx] = [lower_bound, upper_bound]
                    break
            continue  # go to next condition

        # Fallback to single operator logic
        for idx, feature in enumerate(feature_names):
            if feature in condition:
                cond_clean = condition.replace('<=', ' LESS_EQUAL ').replace('>=', ' GREATER_EQUAL ')
                cond_clean = cond_clean.replace('<', ' < ').replace('>', ' > ')
                tokens = cond_clean.split()

                tokens = ['<=' if token == 'LESS_EQUAL' else token for token in tokens]
                tokens = ['>=' if token == 'GREATER_EQUAL' else token for token in tokens]

                operator = None
                operator_pos = None
                for i, token in enumerate(tokens):
                    if token in ['>', '>=', '<', '<=']:
                        operator = token
                        operator_pos = i
                        break

                value = None
                if operator is not None and operator_pos is not None:
                    for i in range(operator_pos + 1, len(tokens)):
                        try:
                            value = float(tokens[i])
                            break
                        except ValueError:
                            continue

                if value is not None:
                    if operator == '>':
                        bounds[idx] = [value + epsilon, 1]
                    elif operator == '>=':
                        bounds[idx] = [value, 1]
                    elif operator == '<':
                        bounds[idx] = [0, value - epsilon]
                    elif operator == '<=':
                        bounds[idx] = [0, value]
                else:
                    print(f"Could not extract numeric value from condition: '{condition}'")

    return np.array(bounds)
def train_anchors(dataset_name):
    print(dataset_name)
    clf = joblib.load(f'models/{dataset_name}_svm_model.pkl')
    print(f'Loaded model')
    X_train, X_test, y_train, y_test,feature_names, class_names = load_data(dataset_name)
    if 'target' in feature_names:
        feature_names = feature_names[feature_names != 'target']
    print(f'feature names:\n {feature_names}')
    print(f'class names:\n {class_names}')
    predict_fn = lambda x: clf.predict(x)
    explainer = AnchorTabular(predict_fn, feature_names)
    explainer.fit(X_train, disc_perc=(25, 50, 75))
    return explainer, feature_names, class_names

In [91]:
dataset_name = 'Sonar' #Iris, Wine, Vertebral-Column, Pima, Parkinsons, Breast_Cancer, Blood_Transfusion, Ionosphere, Glass, Climate, Modeling, Banknote, Sonar
path_anchors = f'./Anchors_results/{dataset_name}_results.csv' 
path_twostep = dataset_name+ '_results/results_0.25.csv'
path_twostep_brute = dataset_name+ '_results/raw_metric_data_0.25.csv'
np.random.seed(50)

In [92]:
anchors_brute_results = pd.read_csv(path_anchors)
twostep_results = pd.read_csv(path_twostep)
twostep_results_brute = pd.read_csv(path_twostep_brute)

In [93]:
twostep_results_brute

,times_onestep,times_twostep,feature_sizes_onestep,feature_sizes_twostep,rsum_onestep,rsum_twostep,coverage_onestep,coverage_twostep,time_relative_%,sizes_relative_%,rsum_relative_%,coverage_relative_%
0,0.192907,0.299260,53,53,32.666765,32.666765,1,1,55.131983,0.0,0.000000,0.0
1,0.194326,0.317852,56,56,32.101711,32.119355,1,1,63.566500,0.0,0.054963,0.0
2,0.174175,0.271793,47,47,34.848134,34.848546,1,1,56.045368,0.0,0.001181,0.0
3,0.188589,0.299771,53,53,33.563811,33.585372,1,1,58.954795,0.0,0.064237,0.0
4,0.165660,0.264267,43,43,34.533158,34.531416,1,1,59.523772,0.0,-0.005044,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
151,0.178652,0.275697,49,49,29.749029,29.750363,1,1,54.320233,0.0,0.004482,0.0
152,0.184798,0.288714,52,52,28.727888,28.729458,1,1,56.232011,0.0,0.005467,0.0
153,0.186537,0.293915,51,51,30.002368,30.007849,1,1,57.564267,0.0,0.018266,0.0
154,0.188110,0.296630,52,52,27.942881,27.951010,1,1,57.689521,0.0,0.029089,0.0


In [94]:
anchors_brute_results

,coverage,errors,time,rsum,size
0,3,0,3.759618,54.660000,3
1,8,3,4.709863,56.880000,3
2,1,0,2.388251,57.539996,2
3,1,0,3.644434,55.979998,3
4,1,0,2.981253,57.539996,2
...,...,...,...,...,...
151,1,0,3.379192,57.019996,3
152,1,0,5.139333,58.089996,3
153,1,0,4.070577,57.519994,3
154,4,0,4.091458,55.730000,3


In [95]:
times_anchors = anchors_brute_results['time'].values
coverage_anchors = anchors_brute_results['coverage'].values
errors_anchors = anchors_brute_results['errors'].values
rsum_anchors = anchors_brute_results['rsum'].values
sizes_anchors = anchors_brute_results['size'].values

times_twostep = twostep_results_brute['times_twostep'].values
coverage_twostep = twostep_results_brute['coverage_twostep'].values
rsum_twostep = twostep_results_brute['rsum_twostep'].values
feature_sizes_twostep = twostep_results_brute['feature_sizes_twostep'].values

In [96]:
# Compute means and standard deviations
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

# Compute relative percentage differences
def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100



# Ensure all lists are NumPy arrays
times_twostep = np.array(times_twostep)
coverage_twostep = np.array(coverage_twostep)
sizes_anchors = np.array(sizes_anchors)
feature_sizes_twostep = np.array(feature_sizes_twostep)
rsum_anchors = np.array(rsum_anchors)
rsum_twostep = np.array(rsum_twostep)



# Compute means and standard deviations
(time_mean_anchors, time_std_anchors) = compute_mean_std(times_anchors)
(time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)

(coverage_mean_anchors, coverage_std_anchors) = compute_mean_std(coverage_anchors)
(coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)

(sizes_mean_anchors, sizes_std_anchors) = compute_mean_std(sizes_anchors)
(sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(feature_sizes_twostep)

(rsum_mean_anchors, rsum_std_anchors) = compute_mean_std(rsum_anchors)
(rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)

# Compute relative percentage differences (Mean & Std)
time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_anchors)
coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_anchors)

time_std_diff = relative_percentage_diff(time_std_twostep, time_std_anchors)
coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_anchors)

sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_anchors)
sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_anchors)

rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_anchors)
rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_anchors)

# Compute pointwise relative differences
time_relative_pointwise = relative_percentage_diff(times_twostep, times_anchors)
coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_anchors)

sizes_relative_pointwise = relative_percentage_diff(feature_sizes_twostep, sizes_anchors)
rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_anchors)

# Compute pointwise means
time_relative_mean = np.mean(time_relative_pointwise) 
coverage_relative_mean = np.mean(coverage_relative_pointwise)
sizes_relative_mean = np.mean(sizes_relative_pointwise)
rsum_relative_mean = np.mean(rsum_relative_pointwise)

# Compute pointwise standard deviations
time_relative_std = np.std(time_relative_pointwise) 
coverage_relative_std = np.std(coverage_relative_pointwise)
sizes_relative_std = np.std(sizes_relative_pointwise)
rsum_relative_std = np.std(rsum_relative_pointwise)

# Organize Data
all_metrics_data = all_metrics_data = {
    'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
    'ANCHORS_MEAN': [time_mean_anchors, sizes_mean_anchors, rsum_mean_anchors, coverage_mean_anchors],
    'ANCHORS_STD': [time_std_anchors, sizes_std_anchors, rsum_std_anchors, coverage_std_anchors],
    'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
    'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
    'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
    'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
    'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
    'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std],
    'ANCHORS_ERROR_RATE_MEAN%': [None, None, None, np.mean(errors_anchors/coverage_anchors)*100],
    'ANCHORS_ERROR_RATE_STD%': [None, None, None, np.std(errors_anchors/coverage_anchors)*100],
    
}
# Display and save
all_metrics_df = pd.DataFrame(all_metrics_data)
display(all_metrics_df)
print(errors_anchors.sum())
all_metrics_df.to_csv(f'Anchors_vs_Twostep_Results/Original_{dataset_name}_results.csv', index=False)

,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE_MEAN%,ANCHORS_ERROR_RATE_STD%
0,Time,4.138793,0.816539,0.297104,0.019707,-92.821480,-97.586524,-92.493932,1.833161,NaN,NaN
1,Size,2.743590,0.436651,51.647436,3.800817,1782.476636,770.447517,1840.277778,391.709665,NaN,NaN
2,Ranges_Sum,57.070382,1.107677,31.856469,2.598200,-44.180381,134.563070,-44.158748,4.679504,NaN,NaN
3,Coverage,3.243590,4.417871,1.000000,0.000000,-69.169960,-100.000000,-32.443540,38.076245,2.075322,7.219884


31


In [9]:
# Load Dataset
#df_artificial = pd.read_csv('datasets/artificial/'+f'{dataset_name}_artificial.csv')
clf = joblib.load(f'models/{dataset_name}_svm_model.pkl')
loaded_bounds = np.load(f'models/{dataset_name}_data_bounds.npz')
lower_bound = loaded_bounds['lower_bound']
upper_bound = loaded_bounds['upper_bound']
np.random.seed(50)

In [10]:
d = 0.5
p = 0.25
num_instances=100
X_test = pd.read_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv')
epsilon = 1e-6
anchors_explainer, feature_names,class_names = train_anchors(dataset_name)
anchors_explainer.save(f'Anchors_explainers/{dataset_name}_model')


Sonar
Loaded model
Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True
feature names:
 ['f1' 'f2' 'f3' 'f4' 'f5' 'f6' 'f7' 'f8' 'f9' 'f10' 'f11' 'f12' 'f13'
 'f14' 'f15' 'f16' 'f17' 'f18' 'f19' 'f20' 'f21' 'f22' 'f23' 'f24' 'f25'
 'f26' 'f27' 'f28' 'f29' 'f30' 'f31' 'f32' 'f33' 'f34' 'f35' 'f36' 'f37'
 'f38' 'f39' 'f40' 'f41' 'f42' 'f43' 'f44' 'f45' 'f46' 'f47' 'f48' 'f49'
 'f50' 'f51' 'f52' 'f53' 'f54' 'f55' 'f56' 'f57' 'f58' 'f59' 'f60']
class names:
 [0 1]


In [11]:
twostep_coverages = []
anchors_coverages = []
anchors_errors = []
anchors_rsums = []
anchors_sizes = []
for instance in X_test.values:
    #print(instance)
    artificial_lower_bounds = instance-d
    artificial_lower_bounds[artificial_lower_bounds<lower_bound] = lower_bound
    artificial_upper_bounds = instance+d
    artificial_upper_bounds[artificial_upper_bounds>upper_bound] = upper_bound
    #print(artificial_lower_bounds,artificial_upper_bounds)
    
    data = np.random.uniform(low=artificial_lower_bounds, high=artificial_upper_bounds, size=(num_instances, len(instance)))
    df_artificial = pd.DataFrame(data, columns=X_test.columns)
    df_artificial.loc[len(df_artificial)] = instance #Add original instance to the artificial set
    X_test_art = df_artificial.values
    test_dataset_df = df_artificial
    test_dataset_df['target'] = clf.predict(df_artificial.values)
    original_instance_df = pd.DataFrame(instance.reshape(1,-1), columns=X_test.columns)
    original_instance_df['target'] = test_dataset_df['target'].values[-1]
    #print(test_dataset_df['target'].unique())
        
    # Finding patterns classified as positive/negative
    positive_indexes,negative_indexes = utility.find_indexes(clf, instance.reshape(1, -1), threshold=0)
    
    #Variables for results
    #coverage_twostep = []
    pos_exp_twostep = []
    neg_exp_twostep = []
    
    #Generate Explanations for the patterns classified as negative
    for idx in  negative_indexes:
        
        #Twostep
        exp_ = docplex_explainer.twostep(
                classifier = clf,
                dual_coef = clf.dual_coef_,
                support_vectors = clf.support_vectors_,
                intercept = clf.intercept_,
                lower_bound = lower_bound,
                upper_bound = upper_bound,
                data = (instance.reshape(1, -1)),
                p = p,
                positive = False)
        neg_exp_twostep.append(exp_)
        #coverage_twostep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp_)))
        twostep_coverages.append(len(mymetrics.calculate_coverage(test_dataset_df.drop(columns=['target']), exp_)))

        #Anchors
        prediction = class_names[anchors_explainer.predictor(instance.reshape(1, -1))[0]]
        explanation = anchors_explainer.explain(instance.reshape(1, -1), threshold=1)
        bounds = parse_explanation(explanation.anchor, feature_names, epsilon)
        rsum = mymetrics.range_sum(bounds)
        size = 0
        for feature_name in feature_names:
            for anchor in explanation.anchor:
                if feature_name in re.split(r' <= | >= | < | > ', anchor):
                    size += 1
        coverage_df = mymetrics.calculate_coverage(test_dataset_df, bounds)
        anchor_coverage = len(coverage_df)
        error = len(coverage_df[coverage_df['target'] != prediction])

        instance_coverage = len(mymetrics.calculate_coverage(original_instance_df, bounds))
        if instance_coverage == 0:
            anchor_coverage += 1

        if anchor_coverage == 0:
            print('WARNING')
        
        anchors_coverages.append(anchor_coverage)
        anchors_errors.append(error)
        anchors_rsums.append(anchors_rsum)
        anchors_sizes.append(anchors_size)
    
    #Generate Explanations for the patterns classfied as positive
    for idx in positive_indexes:
        #Twostep
        exp_ = docplex_explainer.twostep(
                classifier = clf,
                dual_coef = clf.dual_coef_,
                support_vectors = clf.support_vectors_,
                intercept = clf.intercept_,
                lower_bound = lower_bound,
                upper_bound = upper_bound,
                data = (instance.reshape(1, -1)),
                p = p,
                positive = True)
        pos_exp_twostep.append(exp_)
        #coverage_twostep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp_)))
        twostep_coverages.append(len(mymetrics.calculate_coverage(test_dataset_df.drop(columns=['target']), exp_)))

        #Anchors
        prediction = class_names[anchors_explainer.predictor(instance.reshape(1, -1))[0]]
        explanation = anchors_explainer.explain(instance.reshape(1, -1), threshold=1)
        bounds = parse_explanation(explanation.anchor, feature_names, epsilon)
        rsum = mymetrics.range_sum(bounds)
        size = 0
        for feature_name in feature_names:
            for anchor in explanation.anchor:
                if feature_name in re.split(r' <= | >= | < | > ', anchor):
                    size += 1
        coverage_df = mymetrics.calculate_coverage(test_dataset_df, bounds)
        anchor_coverage = len(coverage_df)
        error = len(coverage_df[coverage_df['target'] != prediction])

        instance_coverage = len(mymetrics.calculate_coverage(original_instance_df, bounds))
        if instance_coverage == 0:
            anchor_coverage += 1

        if anchor_coverage == 0:
            print('WARNING')
        
        anchors_coverages.append(anchor_coverage)
        anchors_errors.append(error)
        anchors_rsums.append(anchors_rsum)
        anchors_sizes.append(anchors_size)
        
    
# Compute means and standard deviations
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

# Compute relative percentage differences
def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100

# Ensure all lists are NumPy arrays
anchors_coverages = np.array(anchors_coverages)
twostep_coverages = np.array(twostep_coverages)
anchors_errors = np.array(anchors_errors)

(coverage_mean_anchors, coverage_std_anchors) = compute_mean_std(anchors_coverages)
(coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(twostep_coverages)

coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_anchors)
coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_anchors)
coverage_relative_pointwise = relative_percentage_diff(twostep_coverages, anchors_coverages)
coverage_relative_mean = np.mean(coverage_relative_pointwise)
coverage_relative_std = np.std(coverage_relative_pointwise)

# Organize Data
all_metrics_data = {
    'Metric': ['Coverage'],
    'ANCHORS_MEAN': [coverage_mean_anchors],
    'ANCHORS_STD': [coverage_std_anchors],
    'TWOSTEP_MEAN': [coverage_mean_twostep],
    'TWOSTEP_STD': [coverage_std_twostep],
    'MEAN_DIFF_%': [coverage_mean_diff],
    'STD_DIFF_%': [coverage_std_diff],
    'POINTWISE_MEAN_%': [coverage_relative_mean],
    'POINTWISE_STD_%': [coverage_relative_std],
    'ANCHORS_ERROR_RATE%': np.mean(anchors_errors/anchors_coverages)*100
}
# Display and save
all_metrics_df = pd.DataFrame(all_metrics_data)
display(all_metrics_df)
all_metrics_df.to_csv(f'Anchors_vs_Twostep_Results/Artificial_{dataset_name}_results.csv', index=False)

#Save Raw Metric Data
raw_df = pd.DataFrame({
    "coverage_anchors": anchors_coverages, 
    "coverage_twostep": twostep_coverages,
    "coverage_relative_%": coverage_relative_pointwise
})

display(raw_df)
raw_df.to_csv(f"{dataset_name}_results/Artificial_coverage_raw_metric_data_{p}.csv", index=False)

,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE%
0,Coverage,3.602564,5.504869,1.0,0.0,-72.241993,-100.0,-28.954462,38.099699,3.60777


,coverage_anchors,coverage_twostep,coverage_relative_%
0,1,1,0.000000
1,3,1,-66.666667
2,1,1,0.000000
3,1,1,0.000000
4,1,1,0.000000
...,...,...,...
151,7,1,-85.714286
152,5,1,-80.000000
153,1,1,0.000000
154,1,1,0.000000
